In [ ]:
import sys
import time
import itertools
import threading
import importlib
from automl import AutoML
import util


In [2]:
importlib.reload(util)

<module 'util' from 'c:\\Users\\irene\\Desktop\\Heart_Disease\\util.py'>

In [3]:
# Logger configuration
logger = util.get_logger('PipelineApp', log_file='application.log')

In [4]:
def show_loading_message(message="Processing, please do not close this window"):
    """
    Display a loading message with a rotating spinner in the console.
    
    Args:
        message (str): The message to display.
    """
    spinner = itertools.cycle(['-', '\\', '|', '/'])  # cycle for the wheel
    sys.stdout.write(f"\n{message}... ")  # Initial message
    sys.stdout.flush()
    
    try:
        while True:
            sys.stdout.write(next(spinner))  # Write next symbol
            sys.stdout.flush()
            time.sleep(0.3)  # Pause for animation's speed
            sys.stdout.write('\b')  # Back to previous character
    except KeyboardInterrupt:
        sys.stdout.write("\nOperation interrupted by user.\n")
        sys.stdout.flush()

In [5]:
def run_loading(stop_event):
    # Use print statements or IPython.display to update loading messages for Jupyter
    from IPython.display import display, clear_output
    spinner = itertools.cycle(['-', '\\', '|', '/'])
    loading_message = "Processing, please do not close this window..."
    while not stop_event.is_set():  # Keep spinning until the stop event is set
        display(loading_message + next(spinner))
        time.sleep(0.3)
        clear_output(wait=True)

In [6]:
def main(csv_path, target_column, algorithm, test_size=0.2, random_state=42, cv=5, model_name="model"):
    """
    Function to execute the pipeline in a Jupyter notebook.
    
    Args:
        csv_path (str): Path to the cleaned CSV file
        target_column (str): Target column name
        algorithm (str): ML algorithm to use
        test_size (float): Proportion of test data
        random_state (int): Random state for reproducibility
        cv (int): Number of cross-validation folds
        model_name (str): Name of the resulting model file
    """
    # Create a stop event to signal when to stop the loading spinner
    stop_event = threading.Event()

    # Run the loading spinner in a separate thread
    loading_thread = threading.Thread(target=run_loading, args=(stop_event,))
    loading_thread.daemon = True
    loading_thread.start()

    try:
        # Check if mandatory parameters are provided
        util.check_mandatory_parameters(csv_path, target_column, algorithm)

        # Initialize and train the AutoML model
        automl = AutoML()
        automl.init(csv_path, target_column, algorithm, random_state, model_name)
        results = automl.train_and_evaluate(test_size, random_state, cv)
        
        # Signal the loading thread to stop
        stop_event.set()

        # Display completion message
        print("\nProcess completed successfully! Check the log file 'application.log' for details.")
    
    except Exception as e:
        # Signal the loading thread to stop in case of error
        stop_event.set()
        
        # Handle errors
        print("\nCritical error! Check the log file 'application.log' for details.")
        logger.critical("Critical error in main: %s", str(e))



In [ ]:
# Fill the parameters
csv_path = "csv/heart.csv"  # Set your actual file path here
target_column = "chol"  # Set your target column name
algorithm = "LinearRegression"  # Set the algorithm you want to use

# Call the main function
main(csv_path, target_column, algorithm,test_size=0.3,cv=5,model_name="test")

'Processing, please do not close this window...-'


Process completed successfully! Check the log file 'application.log' for details.
